In [15]:
import pandas as pd

In [16]:
df_users = pd.read_parquet('../datasets/processed/google/reviews_google.parquet.gz')

In [17]:
df_users.head(1)

,user_id,name,time,rating,gmap_id,resp_date,state,sentiment_textblob_resp,sentiment
0,1.089912e+20,Song Ro,2021-01-06 05:12:07.056,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,NaT,California,0.0,2


In [18]:
# Convertir la columna 'time' a tipo datetime
df_users['time'] = pd.to_datetime(df_users['time'])

# Agrupar por usuario (user_id) y obtener el valor más antiguo de 'time' en cada grupo
df_creation = df_users.groupby('user_id')['time'].min().reset_index(name='creation')

# Eliminar las milésimas de segundo de la columna 'time'
df_users['time'] = df_users['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [19]:
# Agrupar por usuario y calcular el promedio de rating
df_avg_rating = df_users.groupby('user_id')['rating'].mean().reset_index()

# Renombrar la columna resultante como 'stars'
df_avg_rating = df_avg_rating.rename(columns={'rating': 'stars'})

In [20]:
# Eliminamos usuarios duplicados
users = df_users['user_id'].unique()

# Unir el resultado con el dataframe original
df_users = df_users.merge(df_creation, on='user_id', how='left')
df_users = pd.merge(df_users, df_avg_rating, on='user_id', how='left')

In [21]:
df_users.head(1)

,user_id,name,time,rating,gmap_id,resp_date,state,sentiment_textblob_resp,sentiment,creation,stars
0,1.089912e+20,Song Ro,2021-01-06 05:12:07,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,NaT,California,0.0,2,2016-09-25 22:42:48.535,4.666667


In [22]:
# Agrupar, contar por usuario
df_users_rev = df_users.groupby('user_id').size().reset_index(name='review_count')

# Convertir la columna 'review_count' a número entero
df_users_rev['review_count'] = df_users_rev['review_count'].astype(int)

# Merge para columna 'review_count'
df_users = df_users.merge(df_users_rev, on='user_id', how='left')

In [23]:
df_users = df_users[['user_id', 'name', 'creation', 'review_count', 'rating']]
df_users.rename(columns={'rating': 'stars'}, inplace=True)

In [24]:
df_users.to_parquet(r'..\datasets\processed\google\user_google.parquet.gz', compression = 'gzip', index=False)

In [25]:
df_users

,user_id,name,creation,review_count,stars
0,1.089912e+20,Song Ro,2016-09-25 22:42:48.535,3,5
1,1.112903e+20,Rafa Robles,2021-02-09 05:47:28.663,1,5
2,1.126404e+20,David Han,2018-09-10 03:52:14.324,7,4
3,1.174403e+20,Anthony Kim,2019-03-07 05:56:56.355,2,5
4,1.005808e+20,Mario Marzouk,2017-05-16 05:01:41.933,1,5
...,...,...,...,...,...
2452767,1.091054e+20,Rydell Rene,2016-09-01 01:03:33.962,2,4
2452768,1.159805e+20,Erik Johnson,2015-08-13 18:22:04.871,31,4
2452769,1.159471e+20,Angad Johar,2014-10-05 17:09:38.598,5,4
2452770,1.179236e+20,Pedro C,2015-12-01 00:52:27.344,8,3
